In [2]:
!wget -qq https://github.com/shenasa-ai/persian-book-recommender-dataset/raw/main/data.zip

In [3]:
!unzip -qq data.zip

In [ ]:
!head sales.csv

In [4]:
import random
import pandas as pd
import numpy as np

sales_data = pd.read_csv('sales.csv')

In [5]:
purchased_data = sales_data[['member_id', 'good_id']].drop_duplicates()

In [6]:
purchased_data.head(10)

,member_id,good_id
0,5,15347
1,8,2833
2,8,8714
3,9,15672
4,12,15001
5,12,2718
6,12,2722
7,12,2721
8,15,698
9,15,1433


In [7]:
purchased_data['factorized_member_id'] = pd.factorize(purchased_data['member_id'])[0]
factorized_to_normal_member_id_mapping = dict(zip(purchased_data['factorized_member_id'], purchased_data['member_id']))
normal_to_factorized_member_id_mapping = dict(zip(purchased_data['member_id'], purchased_data['factorized_member_id']))

In [8]:
purchased_data['factorized_good_id'] = pd.factorize(purchased_data['good_id'])[0]
factorized_to_normal_good_id_mapping = dict(zip(purchased_data['factorized_good_id'], purchased_data['good_id']))
normal_to_factorized_good_id_mapping = dict(zip(purchased_data['good_id'], purchased_data['factorized_good_id']))

In [9]:
purchased_data.head(10)

,member_id,good_id,factorized_member_id,factorized_good_id
0,5,15347,0,0
1,8,2833,1,1
2,8,8714,1,2
3,9,15672,2,3
4,12,15001,3,4
5,12,2718,3,5
6,12,2722,3,6
7,12,2721,3,7
8,15,698,4,8
9,15,1433,4,9


In [10]:
purchased_set = set(zip(purchased_data['factorized_member_id'], purchased_data['factorized_good_id']))
len(purchased_set)

511139

In [11]:
unique_members = purchased_data['factorized_member_id'].unique()
unique_goods = purchased_data['factorized_good_id'].unique()
n_unique_members = len(unique_members)
n_unique_goods = len(unique_goods)

print(n_unique_members, n_unique_goods)

95695 45174


In [12]:
not_purchased_set = set()

while len(not_purchased_set)!=len(purchased_set):
    random_member_id = random.choice(unique_members)
    random_good_id = random.choice(unique_goods)
    random_tuple = (random_member_id, random_good_id)
    if random_tuple not in purchased_set:
        not_purchased_set.add(random_tuple)

In [13]:
x_np = np.array(list(not_purchased_set))
x_p = np.array(list(purchased_set))

X = np.concatenate([x_np, x_p], axis=0)

y = np.zeros(len(X))
y[-len(x_p):] = 1

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

(817822, 2)
(204456, 2)


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_unique_members, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-User")(user_embedding)

product_input = Input(shape=[1], name="Product-Input")
product_embedding = Embedding(n_unique_goods, 5, name="Product-Embedding")(product_input)
product_vec = Flatten(name="Flatten-Product")(product_embedding)

concat = tf.keras.layers.Concatenate(name="Concatenate")([product_vec, user_vec])
Dense1 = tf.keras.layers.Dense(10, activation='relu')(concat)
drop = tf.keras.layers.Dropout(0.5)(Dense1)
out = tf.keras.layers.Dense(1)(drop)

model = Model([user_input, product_input], out)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [16]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define the callback
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=10, restore_best_weights=True
)

history = model.fit([X_train[:,0],
                     X_train[:,1]],
                    y_train, epochs=50,
                    batch_size=1024,
                    validation_split=0.2,
                    callbacks=[reduce_lr],)

Epoch 1/50
639/639 [==============================] - 20s 22ms/step - loss: 0.4623 - accuracy: 0.7689 - val_loss: 0.3862 - val_accuracy: 0.8249 - lr: 0.0100
Epoch 2/50
639/639 [==============================] - 5s 7ms/step - loss: 0.3425 - accuracy: 0.8438 - val_loss: 0.4056 - val_accuracy: 0.8224 - lr: 0.0100
Epoch 3/50
639/639 [==============================] - 5s 8ms/step - loss: 0.2888 - accuracy: 0.8658 - val_loss: 0.4545 - val_accuracy: 0.8148 - lr: 0.0100
Epoch 4/50
639/639 [==============================] - 4s 6ms/step - loss: 0.2568 - accuracy: 0.8766 - val_loss: 0.5270 - val_accuracy: 0.8171 - lr: 0.0100
Epoch 5/50
639/639 [==============================] - 3s 5ms/step - loss: 0.2370 - accuracy: 0.8822 - val_loss: 0.5997 - val_accuracy: 0.8119 - lr: 0.0100
Epoch 6/50
639/639 [==============================] - 4s 6ms/step - loss: 0.2237 - accuracy: 0.8864 - val_loss: 0.6858 - val_accuracy: 0.8100 - lr: 0.0100
Epoch 7/50
639/639 [==============================] - 5s 8ms/step - 

In [18]:
model.evaluate([X_train[:,0],X_train[:,1]], y_train)

25557/25557 [==============================] - 67s 3ms/step - loss: 0.4069 - accuracy: 0.9007


[0.4068845212459564, 0.9007106423377991]

In [19]:
model.evaluate([X_test[:,0],X_test[:,1]], y_test)

6390/6390 [==============================] - 17s 3ms/step - loss: 1.4995 - accuracy: 0.8104


[1.4994908571243286, 0.8104384541511536]